<a href="https://colab.research.google.com/github/sydstewart/Orders/blob/main/Carwash1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:

!pip install simpy


# import simpy
# def example(env):
#    event = simpy.events.Timeout(env, delay=5, value=45)
#    value = yield event
#    print('now=%d, value=%d' % (env.now, value))

# env = simpy.Environment()
# example_gen = example(env)
# p = simpy.events.Process(env, example_gen)
# env.run()
# # now=1, value=42
"""
Software Implementation example.

Covers:

- Waiting for other processes
- Resources: Resource

Scenario:
  A software implementation company has a limited number of configurers and defines
  an implementation processes that takes some (random) time.

  A project arrives at the configurer at a random time. If one configurer is available, they start the implementation process and wait for it
  to finish. If not, the project waits to a configurur is free
"""
import random
import pandas as pd
import simpy

RANDOM_SEED = 42
NUM_CONFIGS = 2  # Number of configurers
# WASHTIME = random.randint( 5,25)      # Minutes it takes to clean a project
T_INTER = 4       # Create a project every ~7 minutes
SIM_TIME = 254    # Simulation time in work=days


column_names = ["a", "b"]
elapsed_time  = []
print(elapsed_time)
k = 0
class Carwash(object):
    """A configurer has a limited number of machines (``NUM_CONFIGS``) to
    clean projects in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_configs, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_configs)
        self.washtime = washtime

    def implementation(self, project):
        """The implementation processes. It takes a ``project`` processes and tries
        to finish it."""
        yield self.env.timeout(WASHTIME)
        print("Configurer removed %d%% of %s's issues." %
              (random.randint(50, 99), project))


def project(env, name, configurer):
    """The project process (each project has a ``name``) arrives at the configurer
    (``configurer``) and requests a cleaning machine.

    It then starts the implemenation process, waits for it to finish and
    leaves to never come back ...

    """
    print('%s arrives at the configurer at %.2f.' % (name, env.now))
    with configurer.machine.request() as request:
        yield request
        enter_time = env.now
        print('%s enters the configurer at %.2f.' % (name, enter_time))
        yield env.process(configurer.implementation(name))
        leave_time = random.randint( 5,25) + enter_time
        time_taken = leave_time - enter_time
        print('%s leaves the configurer at %.2f.' % (name, leave_time))
        elapsed_time.append(time_taken)
        print(elapsed_time)
        

def setup(env, num_configs, washtime, t_inter):
    """Create a configurer, a number of initial projects and keep creating projects
    approx. every ``t_inter`` minutes."""
    # Create the configurer
    configurer = Carwash(env, num_configs, washtime)

    # Create 4 initial projects
    for i in range(2):
        env.process(project(env, 'Car %d' % i, configurer))

    # Create more projects while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        WASHTIME = random.randint( 5,25) 
        i += 1
        env.process(project(env, 'Car %d' % i, configurer))

# Setup and start the simulation
print('Implementation')
print(elapsed_time)

random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_CONFIGS, WASHTIME, T_INTER))
# average_time = (sum(elapsed_time) / len(elapsed_time))
# print('Mean_elapsed_time=',average  )
# Execute!
env.run(until=SIM_TIME)


[]
Implementation
[]
Car 0 arrives at the configurer at 0.00.
Car 1 arrives at the configurer at 0.00.
Car 0 enters the configurer at 0.00.
Car 1 enters the configurer at 0.00.
Car 2 arrives at the configurer at 2.00.
Car 3 arrives at the configurer at 6.00.
Car 4 arrives at the configurer at 9.00.
Car 5 arrives at the configurer at 11.00.
Configurer removed 87% of Car 0's issues.
Configurer removed 77% of Car 1's issues.
Car 6 arrives at the configurer at 13.00.
Car 0 leaves the configurer at 7.00.
[7]
Car 1 leaves the configurer at 11.00.
[7, 11]
Car 2 enters the configurer at 13.00.
Car 3 enters the configurer at 13.00.
Car 7 arrives at the configurer at 15.00.
Car 8 arrives at the configurer at 21.00.
Car 9 arrives at the configurer at 23.00.
Configurer removed 95% of Car 2's issues.
Configurer removed 91% of Car 3's issues.
Car 10 arrives at the configurer at 26.00.
Car 2 leaves the configurer at 25.00.
[7, 11, 12]
Car 3 leaves the configurer at 32.00.
[7, 11, 12, 19]
Car 4 enters